In [ ]:
#!pip install dash

In [ ]:
#!pip install dash_bootstrap_components 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_bootstrap_components as dbc
import dash
from dash import dcc
from dash import html

from dash.dependencies import Input, Output, State


import plotly.offline as pyo
import plotly.graph_objs as go


In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
for i in data.columns:
    print(i,"    ",data[i].dtype)

In [ ]:
data.info()

In [ ]:
data.describe(include='all')

In [ ]:
state=list(data['State'].unique())
state

In [ ]:
data['Year']=pd.DatetimeIndex(data['Ship Date']).year
data['Year']=data['Year'].astype(str)

In [ ]:
data['Year2']=pd.DatetimeIndex(data['Order Date']).year
data['Year2']=data['Year2'].astype(str)

In [ ]:
data['Year']==data['Year2']

In [ ]:
data.info()

In [ ]:
categorydict = data.groupby('Category')['Sub-Category'].apply(list).to_dict()
categorydict

In [ ]:
state_sales=data.groupby(['State'])['Sales'].sum().reset_index()
state_sales

In [ ]:
seg=data.groupby(['Segment','State','Category','Sub-Category','Year'])['Sales'].sum().reset_index()
seg

In [ ]:
##dropdown

dictionary = data.groupby('State')['City'].apply(list).to_dict()
categorydict = data.groupby('Category')['Sub-Category'].apply(list).to_dict()

seg=data['Segment'].unique()
seglist=list(seg)

Year=data['Year'].unique()

In [ ]:
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app=dash.Dash(__name__)
app.config.suppress_callback_exceptions = True


app.layout=html.Div(children=[html.H1('UNITED STATES SALES - STATE WISE ANALYSIS',style={'textAlign':'center','color':'#503D36','font-size':24}),

                              dcc.Graph(id='Statesales',figure=px.bar(state_sales, x='State', y='Sales')),
                              #dbc.CardBody([
                                  html.Label([
                                     html.H3( 'STATE'), html.Br(),dcc.Dropdown(id='state',options=[{'label':i,'value':i} for i in dictionary.keys()],placeholder='select state',)                                
                                  ]),




                                  
                              #],style={"display": "flex", "flexWrap": "wrap", 'align-items':'center'}),
                            
                       
   
                          

     html.Div([  
        html.Div([
            dcc.Graph(id="salespercity")
        ], className="six columns"),
     ], className="row"),
       
  
       
# html.Div([       
html.Label([
    html.Br(),
html.H3('CITY'),html.Br(), dcc.Dropdown(id='city',placeholder='select city',)
]), 
# ]),
       

html.Div([  
        html.Div([
            dcc.Graph(id="salespercategory")
        ], className="six columns"),
    ], className="row"),
                              
html.Label([
html.H3('CATEGORY'),html.Br(),dcc.Dropdown(id='category',options=[{'label':i,'value':i} for i in categorydict.keys()],placeholder='select product category',)
]),
                              

                              
    html.Div([
        html.Div([
            html.H3('Sales Per Year'),
            dcc.Graph(id="salesperyear")
        ], className="six columns"),], className="row"),
           
           
           
html.Label([
html.H3('YEAR'),html.Br(),dcc.Dropdown(id='year',options=[{'label':i,'value':i} for i in Year],placeholder='select year', )
]),

           

     html.Div([
        html.Div([
            html.H3('sales per segment'),
            dcc.Graph(id="salespersegment")
        ], className="six columns"),
        
    ], className="row"),
                              
                              
                              
html.Label([
html.H3('SEGMENT'),html.Br(),dcc.Dropdown(id='segment',options=[{'label':i,'value':i} for i in seglist],placeholder='select segment',)
]),

    

        dcc.Graph(id='products')
                                 
                           
])
# app.css.append_css({
#     'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
# })


#------------------------------------------------------------------------------------------------
@app.callback(
           Output('city','options'),
           [dash.dependencies.Input('state','value')])
def set_cities_options(selected_country):
    return [{'label': i, 'value': i} for i in list(set(dictionary[selected_country]))]



@app.callback(
    Output('salespercity','figure'),
    Input('state','value')
    #State("salespercity", 'children')
)
def spc(state):
    percity = data[data['State']==state]
    avg_sales = percity.groupby(['State','City'])['Sales'].sum().reset_index()
    sales_percity  = px.bar(avg_sales,x='City',y='Sales',color='State',title='Sales State')
    return sales_percity
 
    
@app.callback(
    Output('salespercategory','figure'),
    
    Input('city','value')
)
def sales_category(city):
    cat_city = data[data['City']==city]
    sales = cat_city.groupby(['City','Category'])['Sales'].sum().reset_index()
    sales_percategory  = px.bar(sales,x='Category',y='Sales',color='City')
    return sales_percategory
    

    
@app.callback(
    Output('salesperyear','figure'),
    Input('city','value'),
    Input('category','value')
)  

def sales_peryear(city,category):
        
    
    cat_year = data[(data['City']==city) & (data['Category']==category)]
    saleyear = cat_year.groupby(['City','Category','Year'])['Sales'].sum().reset_index()
    sales_per_year = px.bar(saleyear,x='Year',y='Sales',color='City')
    return sales_per_year


    
@app.callback(
    Output('salespersegment','figure'),
    Input('city','value'),
    Input('category','value'),
    Input('year','value'),
)
def sales_persegment(city,category,year):

    seg_sales=data[(data['City']==city) & (data['Category'] ==category) & (data['Year']==year)]
    saleseg = seg_sales.groupby(['City','Category','Segment','Year'])['Sales'].sum().reset_index()
    sales_per_seg = px.bar(saleseg,x='Segment',y='Sales',color='City')
    return sales_per_seg
    

@app.callback(
    Output('products','figure'),

    Input('city','value'),Input('category','value'),Input('segment','value'),Input('year','value')
)
def csub(city,category,segment,year):
    #subcat = data[data['Category']==category and data['State']==state]
    subcat = data[(data['City']==city) & (data['Category']==category)&(data['Segment']==segment)& (data['Year']==year)] 
    #avg_cat_sales = subcat.groupby(['Category','Sub-Category'])['Sales'].mean().reset_index()
    pro_category = subcat.groupby(['City','Category','Segment','Year','Sub-Category'])['Sales'].sum().reset_index()
    sales_percat = px.bar(pro_category,x='Sub-Category',y='Sales')
    return sales_percat
#----------------------------------------------------------------------------------------------



# pyo.plot(app, filename='dashboard.html')
if __name__ == '__main__':
    app.run_server(debug=False)
    